<a href="https://colab.research.google.com/github/Ravitejaa249/DIP-project/blob/main/Xception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

# List GPUs
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"GPUs detected: {gpus}")
    # Enable dynamic memory allocation (optional but recommended)
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
else:
    print("No GPU detected, running on CPU.")


GPUs detected: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# 1. GPU setup
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"GPUs detected: {gpus}")
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
else:
    print("No GPU detected, running on CPU.")

# 2. (Optional) Use MirroredStrategy to scale across GPUs:
strategy = tf.distribute.MirroredStrategy()

# Configuration
IMAGE_SIZE = (299, 299)
BATCH_SIZE = 32
EPOCHS = 10

def load_and_preprocess_images(directory, label, max_samples=1000):
    # … your existing function …
    pass

# Data loading, splitting, augmentation code unchanged…

with strategy.scope():
    # Build & compile your Xception-based model here
    input_tensor = Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
    base_model = Xception(
        include_top=False,
        weights='imagenet',
        input_tensor=input_tensor,
        pooling='avg'
    )
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Dense(256, activation='relu', kernel_regularizer='l2')(x)
    x = Dropout(0.6)(x)
    predictions = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=input_tensor, outputs=predictions)
    model.compile(
        optimizer=Adam(learning_rate=1e-4),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

# Callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    min_lr=1e-6
)

# Train model
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
    epochs=EPOCHS,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping, reduce_lr],
    class_weight={0: 1, 1: np.sum(y_train == 0)/np.sum(y_train == 1)},
    steps_per_epoch=len(X_train) // BATCH_SIZE
)

# Evaluate
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'\nTest Accuracy: {test_acc:.4f}')

# Unfreeze some layers for fine-tuning
print("Unfreezing layers for fine-tuning...")
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Recompile with lower learning rate
optimizer = Adam(learning_rate=1e-5)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Continue training
history_fine = model.fit(
    datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
    epochs=20,  # Additional fine-tuning epochs
    validation_data=(X_test, y_test),
    callbacks=[early_stopping, reduce_lr],
    class_weight={0: 1, 1: np.sum(y_train == 0)/np.sum(y_train == 1)},
    steps_per_epoch=len(X_train) // BATCH_SIZE
)

# Final evaluation
final_loss, final_acc = model.evaluate(X_test, y_test)
print(f'\nFinal Test Accuracy: {final_acc:.4f}')

# Save model
model.save('deepfake_detection_xception.h5')

# Then your training & fine-tuning code as before…


GPUs detected: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


NameError: name 'datagen' is not defined

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# 1. GPU setup
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"GPUs detected: {gpus}")
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
else:
    print("No GPU detected, running on CPU.")

# 2. Strategy for multi-GPU (if more than one)
strategy = tf.distribute.MirroredStrategy()
print(f"Number of devices: {strategy.num_replicas_in_sync}")

# Configuration
IMAGE_SIZE = (299, 299)
BATCH_SIZE = 32
INITIAL_EPOCHS = 10
FINE_TUNE_EPOCHS = 20
MAX_SAMPLES_PER_CLASS = 1000

def load_and_preprocess_images(directory, label, max_samples=MAX_SAMPLES_PER_CLASS):
    images, labels = [], []
    filenames = os.listdir(directory)
    np.random.shuffle(filenames)
    for fname in filenames[:max_samples]:
        path = os.path.join(directory, fname)
        img = cv2.imread(path)
        if img is None:
            continue
        img = cv2.resize(img, IMAGE_SIZE)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.astype('float32') / 255.0
        images.append(img)
        labels.append(label)
    return np.array(images), np.array(labels)

# 3. Load your data
real_dir = '/content/drive/MyDrive/AI-face-detection-Dataset/real'
fake_dir = '/content/drive/MyDrive/AI-face-detection-Dataset/AI'

print("Loading real images...")
X_real, y_real = load_and_preprocess_images(real_dir, 0)
print("Loading fake images...")
X_fake, y_fake = load_and_preprocess_images(fake_dir, 1)

# Combine and split
X = np.concatenate([X_real, X_fake], axis=0)
y = np.concatenate([y_real, y_fake], axis=0)
print(f"Total samples: {len(y)},  Real: {np.sum(y==0)}, Fake: {np.sum(y==1)}")

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

# 4. Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=[0.8, 1.2],
    horizontal_flip=True,
    zoom_range=0.2
)

# 5. Build & compile model under strategy.scope()
with strategy.scope():
    input_tensor = Input(shape=(*IMAGE_SIZE, 3))
    base_model = Xception(
        include_top=False,
        weights='imagenet',
        input_tensor=input_tensor,
        pooling='avg'
    )
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Dense(256, activation='relu', kernel_regularizer='l2')(x)
    x = Dropout(0.6)(x)
    outputs = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=input_tensor, outputs=outputs)
    model.compile(
        optimizer=Adam(learning_rate=1e-4),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

# 6. Callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    min_lr=1e-6
)

# 7. Initial training
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
    epochs=INITIAL_EPOCHS,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping, reduce_lr],
    class_weight={0:1, 1: np.sum(y_train==0)/np.sum(y_train==1)},
    steps_per_epoch=len(X_train)//BATCH_SIZE
)

# 8. Evaluate baseline
loss, acc = model.evaluate(X_test, y_test, verbose=2)
print(f"Baseline Test Accuracy: {acc:.4f}")

# 9. Fine-tuning: unfreeze last 20 base_model layers
print("Unfreezing last 20 layers for fine-tuning…")
for layer in base_model.layers[-20:]:
    layer.trainable = True

model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history_fine = model.fit(
    datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
    epochs=FINE_TUNE_EPOCHS,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping, reduce_lr],
    class_weight={0:1, 1: np.sum(y_train==0)/np.sum(y_train==1)},
    steps_per_epoch=len(X_train)//BATCH_SIZE
)

# 10. Final evaluation & save
final_loss, final_acc = model.evaluate(X_test, y_test, verbose=2)
print(f"Fine-tuned Test Accuracy: {final_acc:.4f}")

model.save('deepfake_detection_xception.h5')
print("Model saved to deepfake_detection_xception.h5")
